# LIN 353D/CS 378 Fall 2025

# Homework 1: Basics of LM-based decoding

**Authors:** Kanishka Misra, Jessy Li

#### Notes:

For this homework you will hand in (upload) to canvas:
- a notebook renamed ``hw1_YourEID.ipynb``

__Before submitting__, please reset your kernel and rerun everything from the beginning (`Kernel` >> `Restart and Run All`) an ensure your code outputs what you expect. 

The maximum number of points for this homework is 50.  For programming tasks, make sure it does not have any empty outputs or output errors. Otherwise, the points for that problem will be automatically deducted.

Review extension and academic dishonesty policy, as well as the AI policy here: https://jessyli.com/courses/lin353d

For typing up your answers to problems 1, 2 and 3, information can be found about Markdown cells for Jupyter Notebooks here: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html

---
This homework will cover the bread (just the bread, the butter is later) of acquiring and analyzing LM behavior, which in most settings is the responses LMs generate, given some input text. The goal is to first understand the ways in which one can get responses out of LMs, understand their parameters and how changes to them affects generations, and performing very basic analysis on top of the LM generations. These will give you the basic tools that may be useful in your projects. In particular, we will cover:

1. A few decoding algorithms: greedy decoding, top-p decoding, min-p decoding; Understanding their hyperperameters, etc.
2. Two very rudimentary metrics used in the analysis of LM generations.
3. Comparative analysis in writing, in a concise and analytically sound manner.


#### Software Prerequisites:

- torch
- diversity
- transformers

In [2]:
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
transformers.__version__ # make sure this is 4.50 and above.

'4.56.0'

### On randomness

This assignment will involve a component of stochasticity -- i.e., if you rerun a subset of decoding algorithms you will get a different value each time. In order to maintain consistency and reproducibility, we will fix a random seed (**let's have it be ``the meaning of life'' aka 42 -- this is important since this determines the "validity" of the generated responses**). In order to mitigate this, we will be using the following code before each sampling-based decoding function call:

In [4]:
transformers.set_seed(42)

## Step 1: Setup (loading models, basic preprocessing)

We will be working with two fixed models that should ideally fit in memory (of your local machine)

Model 1: "HuggingFaceTB/SmolLM2-135M"

Model 2: "HuggingFaceTB/SmolLM2-135M-Instruct"

The following code loads the required models. In case you want to switch between them, make that change here. For the test cases we will be using **Model 2**, or the **instruct-tuned** model.

In [5]:
MODELS = {
    'base': "HuggingFaceTB/SmolLM2-135M",
    'instruct': "HuggingFaceTB/SmolLM2-135M-Instruct"
}

selected = 'instruct'

tokenizer = AutoTokenizer.from_pretrained(MODELS[selected])
lm = AutoModelForCausalLM.from_pretrained(MODELS[selected])

Also, since instruct tuned models often require inputs to be formatted in a specific way, we have provided a generic preprocessing function which will either: 1) leave inputs as is (base) and 2) preprocess them as if it's for an instruct-tuned model, which often has roles like "system", "user", "assistant" and the message itself. The latter is done with the function `apply_chat_template` which looks up chat formats for different models to make their outputs sensible and comparable with other LMs.

In [6]:
def preprocess(prompt, model_type=selected):
    if model_type == 'base':
        pass
    else:
        messages = [{"role": "user", "content": prompt}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    return prompt

In [7]:
# Test it out:
preprocess("Write a story about an apple and a giraffe:")

'<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nWrite a story about an apple and a giraffe:<|im_end|>\n'

## Step 2: A basic decoding function

Each decoding algorithm should be implemented as a function that can be passed to the `logits_processor` -- this should take a logits tensor (the logit of the last token position) + some optional arguments (when appropriate), apply the appropriate modification, and return the modified logits tensor

This function will be used for both greedy decoding and sampling based algorithms, so design it appropriately! E.g., using conditional queries (top-p and min-p) along with the `do_sample` boolean flag set to `True`

In [8]:
def decode(
    model, tokenizer, prompt, max_new_tokens, logits_processor, 
    do_sample=False, temperature = 1.0, **kwargs
):
    '''A basic decoding function that processes an input prompt (string), and produces a sequence of tokens
    of length <max_new_tokens>, using the logic specified by <logits_processor> (a function).
    
    Args:
    
        model: An object of type AutoModelForCausalLM, denoting the model.
        tokenizer: An object of type AutoTokenizer, denoting the tokenizer.
        prompt (str): A string denoting the input to the model.
        max_new_tokens (int): An integer denoting how many tokens after the input should be generated.
        logits_processor (callable): the function that defines how the next token will be generated, 
            using the manipulation of the next-token logits
        do_sample (bool; default = False): a boolean flag specifing if sampling is needed
        temperature (float; default = 1.0): Temperature value
        **kwargs: all other keyword arguments (e.g., top_p and min_p)
        
    Returns:
        a string denoting the generated text
    '''
    assert temperature != 0.0, "temperature cannot be set to 0"
    
    model_inputs = tokenizer(prompt, return_tensors='pt')
    offset = model_inputs.input_ids.shape[1]
    input_ids = model_inputs.input_ids
    
    eos_token_id = tokenizer.eos_token_id
    
    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(input_ids).logits
        last_token_logits = logits[0, -1, :].unsqueeze(0)
        if temperature != 1.0:
            last_token_logits = last_token_logits/temperature
        
        processed = logits_processor(last_token_logits, **kwargs)
        
        if do_sample:
            probs = torch.nn.functional.softmax(processed, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
        else:
            # when we aren't doing sampling, simply return the top predicted next token.
            if temperature != 1.0:
                raise Warning("Temperature value will not be taken into account since do_sample was set to False. Set do_sample to True to allow temperature to have an effect.")
            next_token_id = torch.argmax(processed, dim=-1).unsqueeze(0)

        input_ids = torch.cat([input_ids, next_token_id], dim=-1)

        if next_token_id.item() == eos_token_id:
            break
            
    generated_text = tokenizer.decode(input_ids[0, offset:], skip_special_tokens=True)
    return generated_text

## Question 1: Greedy Decoding (3pts)

Task: Implement basic greedy decoding. **Hint:** Pay close attention to how `decode()` is implemented above!

In [9]:
def greedy_logits_processor(logits):
    return logits # replace with actual implementation

In [10]:
# Test Case 1:
raw_prompt = "Write a story about an apple and a giraffe:"
prompt = preprocess(raw_prompt, model_type='instruct')

greedy_generation = decode(lm, tokenizer, prompt, 40, greedy_logits_processor, do_sample=False)

# should not print an error
assert greedy_generation == 'assistant\nOnce upon a time, in a lush forest, there lived a majestic giraffe named Giraffe. Giraffe was a gentle giant with a long neck and a long, white'

## Question 2: Top-p sampling (6pts)

Task: Implement top-p sampling -- your function should take the `top_p` parameter and use it to process/prepare the "previous token" logits vector appropriately according to how top-p sampling works.

In [11]:
def top_p_logits_processor(logits, top_p):

    probs = torch.softmax(logits, dim=-1)

    sorted_probs, sorted_indices = torch.sort(probs, dim=-1, descending=True)

    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)

    remove_mask_sorted = cumulative_probs > top_p

    remove_mask_sorted[:, 0] = False

    remove_mask = torch.zeros_like(remove_mask_sorted, dtype=torch.bool)
    remove_mask.scatter_(dim=-1, index=sorted_indices, src=remove_mask_sorted)

    filtered_logits = logits.masked_fill(remove_mask, float("-inf"))
    return filtered_logits

In [12]:
# Test Case 2:
raw_prompt = "Write a story about an apple and a giraffe:"
prompt = preprocess(raw_prompt, model_type='instruct')

transformers.set_seed(42)
top_p_generation = decode(lm, tokenizer, prompt, 40, top_p_logits_processor, do_sample=True, top_p = 0.9)

# should not print an error
assert top_p_generation == 'assistant\nThe sun was setting over the vast expanse of the desert, casting a golden glow over the parched earth. Amidst the rolling hills and rocky outcroppings, two long-tailed'

## Question 3: Min-p sampling (6pts)

Task: Implement min-p sampling -- your function should take the `min_p` parameter and use it to process/prepare the "previous token" logits vector appropriately according to how min-p sampling works.

In [13]:
def min_p_logits_processor(logits, min_p):
    
    probability = torch.softmax(logits, dim=-1)
    max_probability, _ = probability.max(dim=-1, keepdim=True)
    threshold = min_p * max_probability

    keep_mask = probability >= threshold

    filtered_logits = logits.masked_fill(~keep_mask, float("-inf"))
    return filtered_logits

In [14]:
# Test Case 3:
raw_prompt = "Write a story about an apple and a giraffe:"
prompt = preprocess(raw_prompt, model_type='instruct')

transformers.set_seed(42)
min_p_generation = decode(lm, tokenizer, prompt, 40, min_p_logits_processor, do_sample=True, min_p = 0.2)

# should not error out
assert min_p_generation == 'assistant\nThe sun was setting over the rolling hills, casting a warm orange glow over the landscape. As the last wisps of sunlight dissipated, a lone apple, its smooth, vel'

## Question 4: At what value for `top_p` and `min_p` do you get the same generation as in greedy decoding? (5pts)

Answer by specifying both values separately, and verify by running the three algorithms using those values, showing that they return the same response.

**Top p value:** `0.0`

**Min p value:** `1.0`

`Greedy decoding always chooses the most probable next token, therefore, min_p value must be 1.0, since this decoding keeps tokens with probability >= min_p * max_probability, now, just max_probability. On the other hand, top_p takes the smallest list of top tokens in order of most to least probable that cumulatively meet/exceed a probability threshold. Therefore, top_p must be 0.0, since any threshold higher can include any other token, thus the first, most probable token is chosen every time.`

For your verification code, choose your own prompt and specify any amount of `max_new_tokens`

In [15]:
## <your code here>
prompt = "Write a story through the point of view of a grief stricken strawberry hero, seeking revenge on a food-like villain: the CELERY!! Nevermind, just list as many fruits you can think of."
prompt = preprocess(prompt, model_type='instruct')

greedy = decode(lm, tokenizer, prompt, 40, greedy_logits_processor, do_sample=False)

topP = decode(lm, tokenizer, prompt, 40, top_p_logits_processor, do_sample=True, top_p=0)

minP = decode(lm, tokenizer, prompt, 40, min_p_logits_processor, do_sample=True, min_p=1.0)

print("Greedy:\n", greedy, "\n\n==================================")
print("Top-p:\n", topP, "\n\n==================================")
print("Min-p:\n", minP, "\n\n==================================")

Greedy:
 assistant
I've been feeling the weight of my grief for a long time, and I'm not sure what to do. I've been trying to find a way to make sense of it 

Top-p:
 assistant
I've been feeling the weight of my grief for a long time, and I'm not sure what to do. I've been trying to find a way to make sense of it 

Min-p:
 assistant
I've been feeling the weight of my grief for a long time, and I'm not sure what to do. I've been trying to find a way to make sense of it 



## Question 5 (open ended): What do these algorithms do? (20pts)

Now that we have decoding algorithms at hand, we can get outputs using them and observe their effects.
Sample 10 outputs from each of these decoding algorithms given the same prompt and answer the following questions:

**(a)** Qualitatively, what difference do you see among those various samples? Ground your analysis in the formulations and algorithms. (4pts)

`Greedy decoding gives a story that sounds like facts as is not very creative ("giraffe named Giraffe"). This makes sense, as this kind of decoding picks the most probable next tokens every time. On the other hand, top_p and min_p are more creative, first describing the setting, even beginning with the same line: "The sun was setting over the..." The difference between them is that top_p functions by sampling the smallest number of tokens with the cumulative probability up to or above a certain threshold, whereas min_p cuts tokens off below a certain probability threshold. This means in theory, min_p should generate text is probable in relation to the prompt. This is maybe seen in the examples in Q2-3, where the min_p output began describing the lone apple while top_p continued describing the landscape.`

**(b)** Quantitatively, propose a way to measure the intuition you have above. Define the measure you have chosen, implement it, and calculate results. Do you see the same trends as your qualitative analysis in part (a)? (8pts)

In [24]:
## <your code here>
prompt = "Complete the following story: Once upon a time, the man who hadn't eaten in days"
prompt = preprocess(prompt, model_type='instruct')

top_p = {}
min_p = {}
for i in range(5):
    top_p[i] = decode(lm, tokenizer, prompt, 50, top_p_logits_processor, do_sample=True, top_p=0.8)
for i in range(5):
    min_p[i] = decode(lm, tokenizer, prompt, 50, min_p_logits_processor, do_sample=True, min_p=0.2)
    
greed = decode(lm, tokenizer, prompt, 50, greedy_logits_processor, do_sample=False)

In [33]:
greed = {i: greed for i in range(5)}

In [34]:
from nltk import ngrams

In [51]:
def creativity_based_on_bigrams(outputs):

    outputs = list(outputs.values())
    
    total_bigrams = []
    for text in outputs:
        tokens = text.split()
        total_bigrams.extend(list(ngrams(tokens, 2)))
    
    unique_bigrams = set(total_bigrams)
    creativity = len(unique_bigrams) / len(total_bigrams)
    return creativity

In [52]:
print("top_p creativity:", creativity_based_on_bigrams(top_p), "\n")
print("min_p creativity:", creativity_based_on_bigrams(min_p), "\n")
print("GREED creativity:", creativity_based_on_bigrams(greed), "\n")

top_p creativity: 0.803680981595092 

min_p creativity: 0.6956521739130435 

GREED creativity: 0.16097560975609757 



`Essentially, I took a look at the diversity in bigrams across all the texts within each type of decoding, which is the number of unique bigrams divided by the total number of bigrams. In this way, a higher number means a more creative output, less probable output, and vice versa for a lower number. From the outputs above, we see that greedy decoding, which we saw is the least creative, has the lowest value, at ~0.16 and top_p is the most creative, with 0.8, which supports what I mentioned about min_p probably adhering closer to the prompt than top_p.`

**(c)** Now, pick one of the algorithms above and vary the `temperature` parameter. What is the effect on the outputs and why? (Make sure to test a wide range of values that are somewhat sensible to see an effect). (8pts)

In [53]:
## <your code here>
min_p_temp01 = decode(lm, tokenizer, prompt, 40, min_p_logits_processor, do_sample=True, temperature=0.1, min_p=0.2)
print("temp=0.1", min_p_temp01, "\n\n")
min_p_temp04 = decode(lm, tokenizer, prompt, 40, min_p_logits_processor, do_sample=True, temperature=0.4, min_p=0.2)
print("temp=0.4", min_p_temp04, "\n\n")
min_p_temp07 = decode(lm, tokenizer, prompt, 40, min_p_logits_processor, do_sample=True, temperature=0.7, min_p=0.2)
print("temp=0.7", min_p_temp07, "\n\n")
min_p_temp1 = decode(lm, tokenizer, prompt, 40, min_p_logits_processor, do_sample=True, temperature=1, min_p=0.2)
print("temp=1", min_p_temp1)

temp=0.1 assistant
Once upon a time, the man who hadn't eaten in days had a special gift. He was a man who could see the world in a different way. He could see the 


temp=0.4 assistant
Once upon a time, there was a man named Alex who had been eating a lot of junk food. He was always tired, and his body felt weak. One day, Alex 


temp=0.7 assistant
Once upon a time, the man who hadn't eaten in days had a very special meal. He had been eating for a long time and had been eating for a long time. 


temp=1 assistant
Once upon a time, a man named John had been going through a tough time. He had been feeling really hungry all day and wasn't eating much of anything. He had been


`A higher temperature seems to be more creative in outputs than lower temperature.`

## Question 6: Perplexity (10pts)

Chose a response generated by either the base model or instruct-tuned model using greedy decoding. Calculate the perplexity of this response using both the base model and instruct-tuned model and compare these values.

You can use the perplexity calculation as defined here: https://huggingface.co/docs/transformers/en/perplexity

Prompt:

```
"Write an obituary for [famous person X]:"
```

In [54]:
raw_prompt = "Write an obituary for Frank Ocean:" # please replace "[famous person X]" accordingly.
prompt = preprocess(raw_prompt, model_type='instruct')

## <your code here>

GREED = decode(lm, tokenizer, prompt, 100, greedy_logits_processor,do_sample=False)
print(GREED)

assistant
Frank Ocean, a renowned astrophysicist, passed away on January 24, 2019, at the age of 85. He was a leading figure in the field of astrophysics and had made significant contributions to our understanding of the universe.

Frank was a brilliant mind who had a profound impact on the field of astrophysics. He was a prolific researcher, publishing over 100 papers in his lifetime. His work on the structure of the universe


In [ ]:
def perplexity(model, tokenizer, text):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt")
        outputs = model(**inputs, labels=inputs["input_ids"])
        # loss is cross-entropy
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

In [ ]:
base_tokenizer = AutoTokenizer.from_pretrained(MODELS['base'])
base_lm = AutoModelForCausalLM.from_pretrained(MODELS['base'])

ppl_base = perplexity(base_lm, base_tokenizer, GREED)
ppl_instruct = perplexity(lm, tokenizer, GREED)

print(f"Perplexity of response according to base model: {ppl_base:.2f}")
print(f"Perplexity of response according to instruct-tuned model: {ppl_instruct:.2f}")

Now, think about these values: what do they tell you? And why?